In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import date
from bs4 import BeautifulSoup, Comment

In [2]:
today = date.today()

# dd/mm/YY
current_year = today.strftime("%Y")

last_year = int(current_year) - 1

In [3]:
last_five_years = []
for i in range(0,5):
    last_five_years.append(int(current_year) - i)

In [4]:
# Create a list to help create a dataframe from batter statistics data
nba_stats = []

# Create a loop to create a dataframe from Basketball Reference tables
for year in last_five_years:
    
    # input URL and use BeautifulSoup to parse through the page
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_totals.html'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    # Grab the table element that has NBA player statistics
    table = soup.select_one('#div_totals_stats') 

    # Grab data from table and put it into the list created above
    for tr in table.select('tr:has(td)'):
        tds = [td.get_text(strip=True) for td in tr.select('td')]
        tds.append(year)
        nba_stats.append(tds)

In [5]:
# Create dataframe for batter statistics
nba_stats_df = pd.DataFrame(nba_stats)

# Create an empty list to store dataframe header information
header_list = []

# Grab the table header information to use as column headers in our dataframe
for tr in table.select('tr:has(th)'):
    ths = [th.get_text(strip=True) for th in tr.select('th')]
    header_list.append(ths)

# For loop returns a list of lists, and we only need the first list 
df_headers = header_list[0]

# Remove the first item from our headers list, it is the index header that we do not need
df_headers.remove('Rk')
df_headers.append("Year")

# Set column headers equal to our list
nba_stats_df.columns = df_headers

In [6]:
# Change types of columns to numeric for columns with number values
nba_stats_df[['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']] = nba_stats_df[['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].apply(pd.to_numeric)

# Drop all players with NaN games to remove null values
nba_stats_df.dropna(subset=['G'], axis = 0 , inplace= True)

# Remove any players with fewer than 27 games played (1/3rd of the season)
filtered_nba_stats_df = nba_stats_df[nba_stats_df['G'] >= 27]

# points, rebounds, assists, steals, blocks, threes, field-goal percentage, and free-throw percentage

# Select the columns we want for our NBA player analysis
yahoo_nba_df = filtered_nba_stats_df[['Year','Player','Age','Pos','Tm','G','PTS','FG','FGA','FG%','3P','FT','FTA','FT%','TRB','AST','STL','BLK','TOV']]


In [7]:
# Sort by index to prepare to drop duplicates
yahoo_nba_df = yahoo_nba_df.sort_index()

# Drop duplicate entries of Player Name and Year
# This is to eliminate partial season data for players who played for 2+ teams in one season
yahoo_nba_df = yahoo_nba_df.drop_duplicates(subset=['Year', 'Player'])

In [8]:
yahoo_nba_df['FPTS'] = ''
yahoo_nba_df['AVG_FPTS'] = ''

for index, row in yahoo_nba_df.iterrows():
    yahoo_nba_df['FPTS'] = (yahoo_nba_df['PTS'] + yahoo_nba_df['FG'] - yahoo_nba_df['FGA'] + yahoo_nba_df['FT'] - yahoo_nba_df['FTA'] + yahoo_nba_df['TRB'] + yahoo_nba_df['AST'] + (1.5 * yahoo_nba_df['STL']) + (1.5 * yahoo_nba_df['BLK']) - yahoo_nba_df['TOV'])
    yahoo_nba_df['AVG_FPTS'] = (yahoo_nba_df['FPTS']/yahoo_nba_df['G'])


In [9]:
# Sort data by name alphabetically, then by year in descending order
final_yahoo_nba_df = yahoo_nba_df.sort_values(['Year','Player'], ascending=[True, True])


In [10]:
# Create a new dataframe for stats percentile calculations
percentile_df = pd.DataFrame(columns = ['Year', 'Player', 'Age', 'Pos', 'Tm', 'FPTS_Percentile', 'AVG_FPTS_Percentile'])

# Carry over your non-numeric columnns from final_yahoo_nba_df
percentile_df['Year'] = final_yahoo_nba_df['Year']
percentile_df['Player'] = final_yahoo_nba_df['Player']
percentile_df['Age'] = final_yahoo_nba_df['Age']
percentile_df['Pos'] = final_yahoo_nba_df['Pos']
percentile_df['Tm'] = final_yahoo_nba_df['Tm']

# Calculate the percentile rank for each player in each season, seperately, then add all the seasons in one dataframe
for year in last_five_years:
    year_df = final_yahoo_nba_df.loc[final_yahoo_nba_df['Year'] == year]
    year_df['FPTS_Percentile'] = year_df['FPTS'].rank(pct=True)
    year_df['AVG_FPTS_Percentile'] = year_df['AVG_FPTS'].rank(pct=True)
    year_df.sort_values('Player', ascending=True)

    # Each of the seasons are added back to the percentile dataframe
    percentile_df = percentile_df.append(year_df, ignore_index=True)


/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_7326/4104242848.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['FPTS_Percentile'] = year_df['FPTS'].rank(pct=True)
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_7326/4104242848.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['AVG_FPTS_Percentile'] = year_df['AVG_FPTS'].rank(pct=True)


In [11]:
# Clean up the percentile dataframe, drop NaNs and remove unnecessary columns
percentile_df = percentile_df.dropna()
#percentile_df = percentile_df.drop(['Tm','G','PTS','FG','FGA','FG%','3P','FT','FTA','FT%','TRB','AST','STL','BLK','TOV'], axis = 1)

# Add a rank column that adds the percentiles from each category
# Average out percentile categories for FG% and FT% with PTS, as we might still want a player with somewhat  
## lower FG% and FT% if they score a lot of points
percentile_df['Rank'] = (percentile_df['FPTS_Percentile'] + percentile_df['AVG_FPTS_Percentile'])

percentile_df


,Year,Player,Age,Pos,Tm,FPTS_Percentile,AVG_FPTS_Percentile,G,PTS,FG,...,FTA,FT%,TRB,AST,STL,BLK,TOV,FPTS,AVG_FPTS,Rank
1978,2022,Aaron Gordon,26,PF,DEN,0.850962,0.795673,75.0,1126.0,434.0,...,230.0,0.743,439.0,188.0,44.0,44.0,133.0,1293.0,17.240000,1.646635
1979,2022,Aaron Holiday,25,PG,TOT,0.346154,0.252404,63.0,400.0,151.0,...,68.0,0.868,122.0,153.0,42.0,9.0,67.0,488.5,7.753968,0.598558
1980,2022,Aaron Nesmith,22,SF,BOS,0.086538,0.045673,52.0,196.0,72.0,...,26.0,0.808,89.0,22.0,20.0,5.0,31.0,198.5,3.817308,0.132212
1981,2022,Aaron Wiggins,23,SG,OKC,0.3125,0.362981,50.0,416.0,156.0,...,85.0,0.729,178.0,68.0,30.0,10.0,54.0,464.0,9.280000,0.675481
1982,2022,Admiral Schofield,24,SF,ORL,0.064904,0.084135,38.0,146.0,54.0,...,15.0,0.800,89.0,25.0,4.0,5.0,21.0,174.5,4.592105,0.149038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3951,2018,Yogi Ferrell,24,SG,DAL,0.618421,0.492105,82.0,838.0,311.0,...,103.0,0.796,249.0,201.0,64.0,9.0,78.0,879.5,10.725610,1.110526
3952,2018,Zach Collins,20,C,POR,0.239474,0.152632,66.0,292.0,115.0,...,42.0,0.643,221.0,52.0,17.0,31.0,58.0,390.0,5.909091,0.392105
3953,2018,Zach Randolph,36,PF,SAC,0.639474,0.765789,59.0,857.0,361.0,...,107.0,0.785,397.0,127.0,42.0,10.0,116.0,918.0,15.559322,1.405263
3954,2018,Zaza Pachulia,33,C,GSW,0.460526,0.426316,69.0,373.0,149.0,...,93.0,0.806,321.0,109.0,38.0,17.0,72.0,680.5,9.862319,0.886842


In [12]:
# Create a list of each unique player we have in our dataframe
player_list = percentile_df.Player.unique().tolist()

# Create a new dataframe for combined, averaged percentiles over the past 5 seasons
new_df = pd.DataFrame(columns = ['Name', 'Rank', 'Trend', 'Pos', 'Years', 'FPTS', 'AVG_FPTS', 'FPTS_Percentile', 'AVG_FPTS_Percentile'])

# Update new dataframe with unique player list
new_df['Player'] = player_list

# Create a list for each percentile stat category for upcoming loop
player_trends = []
average_FPTS = []
average_AVG_FPTS = []
average_FPTS_Percentile = []
average_AVG_FPTS_Percentile = []
average_Rank = []
year_count = []
pos = []

# Loop through each player, locate their percentile stats for each season, average them out
for player in player_list:
    player_df = percentile_df.loc[percentile_df['Player'] == player]
    
    # We want to find the slope of the line of best fit for each player's overall ranking each season
    x = np.array(player_df['Year'], dtype = float)
    y = np.array(player_df['Rank'], dtype = float)
    slope, intercept = np.polyfit(x, y, 1)
    player_trends.append(slope)
    
    # Find average of each player's percentiles from previous 5 seasons
    average_FPTS.append(sum(player_df['FPTS']) / len(player_df['FPTS']))
    average_AVG_FPTS.append(sum(player_df['AVG_FPTS']) / len(player_df['AVG_FPTS']))
    average_FPTS_Percentile.append(sum(player_df['FPTS_Percentile']) / len(player_df['FPTS_Percentile']))
    average_AVG_FPTS_Percentile.append(sum(player_df['AVG_FPTS_Percentile']) / len(player_df['AVG_FPTS_Percentile']))
    average_Rank.append(sum(player_df['Rank']) / len(player_df['Rank']))
    year_count.append(len(x))
    
    # Keep player positions for reference purposes during the draft
    pos.append(player_df['Pos'].unique())

# Update new dataframe with the list data from each stat
new_df['Pos'] = pos
new_df['Trend'] = player_trends
new_df['FPTS'] = average_FPTS
new_df['AVG_FPTS'] = average_AVG_FPTS
new_df['FPTS_Percentile'] = average_FPTS_Percentile
new_df['AVG_FPTS_Percentile'] = average_AVG_FPTS_Percentile
new_df['Rank'] = average_Rank

# Keep track of how many seasons are being considered, so we know how reliable the data is
new_df['Years'] = year_count


/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

In [13]:
# Create a weighted rank column by adding trend data to the rank data and account for number of seasons played
# Basically, if you played all 5 seasons, your trend stat is added directly
# If you played fewer than all 5 seasons, your trend stat is reduced depending on how few seasons you played
new_df['Weighted Rank'] = (new_df['Rank'] + ((new_df['Trend'] * (new_df['Years'] - 1) / 4)))

# shift column 'Weighted Rank' to first position
first_column = new_df.pop('Weighted Rank')
  
# insert column using insert(position,column_name,first_column) function
new_df.insert(1, 'Weighted Rank', first_column)

new_df = new_df.sort_values('Weighted Rank', ascending = False)

new_df = new_df.round({'FPTS': 0, 'AVG_FPTS': 2})

new_df.head(50)

,Player,Weighted Rank,Rank,Trend,Pos,Years,FPTS,AVG_FPTS,FPTS_Percentile,AVG_FPTS_Percentile
143,Giannis Antetokounmpo,1.995816,1.993863,0.001954,[PF],5,2364.0,35.01,0.995890,0.997973
312,Nikola Jokić,1.991683,1.980223,0.011460,[C],5,2438.0,32.67,0.993753,0.986470
272,Luka Dončić,1.967504,1.937019,0.040647,"[PG, SG]",4,1869.0,28.52,0.967073,0.969946
202,Joel Embiid,1.958974,1.945233,0.013741,[C],5,1854.0,31.05,0.961305,0.983928
349,Rudy Gobert,1.954298,1.928336,0.025962,[C],5,1882.0,27.35,0.962506,0.965830
180,James Harden,1.953151,1.966870,-0.013720,"[PG, PG-SG, SG]",5,2104.0,32.09,0.976784,0.990086
490,Zion Williamson,1.940741,1.940741,0.000480,[PF],1,1728.0,28.33,0.972840,0.967901
228,Karl-Anthony Towns,1.933077,1.940549,-0.007472,[C],5,1933.0,30.47,0.955591,0.984958
266,LeBron James,1.931691,1.944553,-0.012862,"[PF, PG, SF]",5,1930.0,31.36,0.957598,0.986954
106,Domantas Sabonis,1.927178,1.843736,0.083442,"[PF-C, PF, C]",5,1607.0,24.50,0.925159,0.918577


In [14]:
new_pos_list = []
pos_list = new_df['Pos'].tolist()
for item in pos_list:
    new_string = []
    for pos in item:
        string = str(pos)
        new_string = f'{new_string},{string}'
    new_pos_list.append(new_string)

final_pos_list = []
for i in new_pos_list:
    i = i.replace('[],', '')
    final_pos_list.append(i)
    
final_pos_list

new_df.drop('Pos', axis = 1, inplace = True)
new_df['Pos'] = final_pos_list

new_df

,Player,Weighted Rank,Rank,Trend,Years,FPTS,AVG_FPTS,FPTS_Percentile,AVG_FPTS_Percentile,Pos
143,Giannis Antetokounmpo,1.995816,1.993863,0.001954,5,2364.0,35.01,0.995890,0.997973,PF
312,Nikola Jokić,1.991683,1.980223,0.011460,5,2438.0,32.67,0.993753,0.986470,C
272,Luka Dončić,1.967504,1.937019,0.040647,4,1869.0,28.52,0.967073,0.969946,"PG,SG"
202,Joel Embiid,1.958974,1.945233,0.013741,5,1854.0,31.05,0.961305,0.983928,C
349,Rudy Gobert,1.954298,1.928336,0.025962,5,1882.0,27.35,0.962506,0.965830,C
...,...,...,...,...,...,...,...,...,...,...
661,Marcus Georges-Hunt,0.010526,0.010526,0.000003,1,53.0,1.26,0.005263,0.005263,SG
267,Leandro Bolmaro,0.009615,0.009615,0.000002,1,71.0,2.03,0.004808,0.004808,SF
453,Juwan Morgan,0.007407,0.007407,0.000002,1,56.0,1.91,0.002469,0.004938,PF
523,Malcolm Miller,0.005277,0.005277,0.000001,1,46.0,1.66,0.002639,0.002639,SF


In [15]:
draft_df = new_df

In [16]:
######################################################################################################################
######################################################################################################################
######################################################################################################################
#### DRAFT DAY FUNCTIONS
   
# DROP A PLAYER 
def drafted(player):
    global draft_df
    draft_df = draft_df[draft_df.Player != player]
    return draft_df.head(25)
    
# FILTER PLAYERS BY POSITION
def position_filter(Pos):
    filtered_draft_df = draft_df[draft_df['Pos'].str.contains(Pos)]
    return filtered_draft_df.head(25)

# PULL STAT CATEGORY LEADERS
def stat_leaders(CAT):
    global final_yahoo_nba_df
    final_yahoo_nba_df = final_yahoo_nba_df.sort_values([CAT], ascending=[False])
    return final_yahoo_nba_df.head(25)